## Notes on choosing camera position

$\newcommand{\abs}[1]{\left\vert #1 \right\vert}$
$\newcommand{\magn}[1]{\left\Vert #1 \right\Vert}$

$\newcommand{\vec}[1]{\underline{\mathbf{#1}}}$
$\newcommand{\mat}[1]{\underline{\underline{\mathbf{#1}}}}$

$\newcommand{\JJJ}{\mat{J}}$
$\newcommand{\bbb}{\vec{b}}$
$\newcommand{\ccc}{\vec{c}}$
$\newcommand{\rrr}{\vec{r}}$
$\newcommand{\uuu}{\vec{u}}$
$\newcommand{\vvv}{\vec{v}}$
$\newcommand{\beq}{\qquad\begin{align}}$
$\newcommand{\eeq}{\end{align}}$

The `threed` package does the following calculations behind the scenes.

**Back vector.**
Given an elevation $\alpha=\frac{\pi}{2}-\theta$ (measured in radians upwards from the horizontal) and an azimuth angle $\phi$ (measured in radians counterclockwise from the $+x$ axis), calculate the unit vector in this direction,

$\qquad$$\bbb = (\cos\alpha\cos\phi, \cos\alpha\sin\phi, \sin\alpha)$.

In the context of camera orientation, this unit vector defines the camera's *back* vector, which represents the direction in which light travels, which points *opposite* to the direction the camera is facing.  For example, if $\alpha=0$ and $\phi=0$, then $\bbb=(1,0,0)$, so the camera is looking in the $-x$ direction.  If $\alpha=90^\circ$, then $\bbb=(0,0,1)$, so the camera is looking directly downward along the $-z$ direction.

**Camera orientation**.  Given an elevation $\alpha$, azimuth $\phi$, and *vertical* vector $\vvv$, `threed.chooseCameraOrientation (e, a, up=[0., 0., 1.])` returns a 3x3 unimodular orthogonal matrix $\JJJ=(\bbb,\rrr,\uuu)$ whose column vectors are the *right*, *up*, and *back* vectors that define camera space, calculated as follows:
$\newcommand{\abs}[1]{\left\Vert #1 \right\Vert}$

$\beq
\bbb &:= \bbb / \magn{ \bbb } \\
\rrr &:= \vvv \times \ccc / \magn{ \vvv \times \ccc } \\
\uuu &:= \bbb \times \rrr / \magn{ \bbb \times \rrr } \\
\eeq$


**Camera position**.  Some of the calculations below are implemented in `threed.chooseCameraPosition (e, a, up=[0., 0., 1.])`.  Given the objects in a scene, `threed` tries to find a set of points that must lie within view of the camera.  At the moment, it does so by compiling a list of the positions of each Mesh.  (This is not foolproof because a sphere of radius 1 extends 1 unit in each direction beyond its nominal position.)  These points, $\rrr_n$, are in world coordinates.  The points are transformed to camera space: $\uuu_n = \JJJ ~ \rrr_n$.  (Note that the NumPy implementation of this equation looks different due to conventions.)  The coordinates of the points are unpacked: $\uuu_n = (u_n, v_n, w_n)$.  The corners of the bounding box in camera space are calculated:

$\beq
u_\mathrm{min} &= \min_n (u_n)  &v_\mathrm{min} &= \min_n (v_n)  &w_\mathrm{min} &= \min_n (w_n) \\
u_\mathrm{max} &= \max_n (u_n)  &v_\mathrm{max} &= \max_n (v_n)  &w_\mathrm{max} &= \max_n (w_n) .
\eeq$

The camera's target position is chosen to be the center of the bounding box:

$\beq
\uuu_\mathrm{target} &= \left(
\frac{u_\mathrm{min}+u_\mathrm{max}}{2},
\frac{v_\mathrm{min}+v_\mathrm{max}}{2},
\frac{w_\mathrm{min}+w_\mathrm{max}}{2}
\right)
\eeq$
$\beq
\rrr_\mathrm{target} &= \JJJ^{-1} ~ \uuu_\mathrm{target}.
\eeq$

For simplicity, consider the *front* corners of the bounding box.  That is, consider the rectangle 

$\beq
u &\in [u_\mathrm{min},u_\mathrm{max}] \\
v &\in [v_\mathrm{min},v_\mathrm{max}] \\
w &= w_\mathrm{\max}.
\eeq$

We are given the vertical field-of-view of the camera $\beta$ (the angle between the top and bottom viewing directions, typically $50^\circ$) and the aspect ratio of the camera's viewport, $\gamma = Y/X$ (typically $4/3$).
From trigonometry, in order to capture the left and right sides of the rectangle, the cameraman must "step back" from the rectangle by a distance 

$\beq
w_1 &= \abs{u_\mathrm{max} - u_\mathrm{min}} / \left( 2 \gamma \tan \frac{\beta}{2} \right) \\
w_2 &= \abs{v_\mathrm{max} - v_\mathrm{min}} / \left( 2 \tan \frac{\beta}{2} \right) .
\eeq$
 
The correct step-back distance is the larger of these two: $w_0 = \max(w_1, w_2)$.  Thus, to find the correct position of the camera, start at the target position and step back a distance $w_0 + \frac{w_\mathrm{\max} - w_\mathrm{\min}}{2}$ along the camera's *back* direction:

$\beq
\rrr_\mathrm{cam} &= \rrr_\mathrm{target} 
+  \left( w_0 + \frac{w_\mathrm{\max} - w_\mathrm{\min}}{2}  \right)
\bbb
\eeq$


**Rendering.**  `threed.render` performs the following:
- Call `chooseCameraOrientation` if not specified by user.
- Call `chooseCameraPosition` if not specified by user.
- Create an ambient light and a directional light.
- Create a `pythreejs.Camera` containing the objects to draw.  Attach the directional light to the camera.
- Create a `pythreejs.Scene` containing the objects to draw.  Attach the camera to the scene, and attach the ambient light to the scene.
- Create a `pythreejs.OrbitControls`.  Attach this to the camera.
- Set the camera orientation.  (This must be done after attaching the OrbitControls.)
- Create a `pythreejs.Renderer`.  Return this.

In [22]:
# np.set_printoptions (precision=2,floatmode='fixed',suppress=True)
# for pointA,pointB in itertools.combinations (points, 2):
#   if np.linalg.norm(pointB - pointA) < 5.01:
#     objects.append ( threed.cylinder (pointA, pointB, radius=.2, color='#99FF99') )